In [1]:
from tkinter import Tk,StringVar,Label,Entry,Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import sys
import time 
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage



uNoList=set()
report=[]
changDict={"4":"加保","2":"退保","3":"調保"}
web='https://edesk.bli.gov.tw/cpa/'
    
with open('Email.txt') as f:
    mails=f.read().split('\n')
while "" in mails:
    mails.remove("")
mails=set(mails)

def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()        
    
    


def doAddPopChangeAssure():
    report = []
    path2=e3.get()
    unoNumber=0
    excelFilePath=""
    def renamefile(path):
#         pathList2=[i[0]+"/"+j for i in os.walk(path) for j in i[2]]
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'xls' in file_ and 'crdownload' not in file_:
                        name=[]
                        book = xlrd.open_workbook(os.path.join(pathTuple[0], file_))
                        sheet=book.sheet_by_index(0)

                        #日期
                        dateCell=sheet.cell_value(rowx=3,colx=8) or sheet.cell_value(rowx=3,colx=4)
                        if len(dateCell.split("／"))<3:
                            name.append(dateCell.split("：")[1].split(" ")[0])
                        else:
                            name.append(dateCell.split("／")[2] or dateCell.split("／")[1].split("：")[1].split(" ")[0])

                        ##事業部
                        if "公司" in sheet.cell_value(rowx=4,colx=0):
                            name.append(sheet.cell_value(rowx=4,colx=0).split("公司")[1] or sheet.cell_value(rowx=3,colx=0).split("：")[1])
                        else:
                            name.append(sheet.cell_value(rowx=3,colx=0).split("公司")[1] or sheet.cell_value(rowx=2,colx=0).split("：")[1])


                        ##加退調保
                        if '加' in sheet.cell_value(rowx=0,colx=0):
                            name.append("加")
                        elif '退' in sheet.cell_value(rowx=0,colx=0):
                            name.append("退")
                        elif '調' in sheet.cell_value(rowx=0,colx=0):
                            name.append("調")

                        ##二、三合一
                        if '健' in sheet.cell_value(rowx=1,colx=0):
                            name.append("3")
                        else:
                            name.append("2")

                        try:
                            os.rename(os.path.join(pathTuple[0],file_),os.path.join(pathTuple[0],"_".join(name)+".xls"))
                        except:
    #                         print(os.path.join(path, file_)+"\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0],file_))
                except:
                    report.append("更名失敗>>{}".format(os.path.join(pathTuple[0],file_)))


    try:
        uNoList=set()
        report=[]
        b=time.time()
        errorMsg=""
        pinNum=e2.get() #pin號碼
        chrome_options = Options()
        #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
        chrome_options.add_extension('extension_0_0_1_3.crx')
		
		
		##################0516
        # chrome_options.add_argument("--headless")
        # chrome_options.add_argument("--no-sandbox")
        # chrome_options.add_argument("start-maximized")
        # chrome_options.add_argument("disable-infobars")
        # chrome_options.add_argument("--disable-extensions")
		###################0516
        chrome_options.add_experimental_option("prefs", {
          "download.default_directory": e3.get(),
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        
        #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        time.sleep(0.5)
        pa=0
        n=0
        while pa==0:
            try:
                driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
                time.sleep(0.5)
                driver.get(web)
                pa=1
            except:
                time.sleep(0.3)
                n+=1
                if n%20==0:
                    break


        dataDir=e1.get() #需改成人工輸入
        #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
        # for i in os.walk(dataDir):
        #     for j in i[2]:
        #         print(i[0]+j)
        #可縮寫成這樣     
        pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
        myxlobject= xl2dict.XlToDict()
        fileCount=0
        for excelFilePath in pathList:
            try:
                #得到excel內容成dict
                dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
                time.sleep(0.2)
                #進入勞保局
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                time.sleep(0.4)
            #     alert=driver.switch_to_alert()
            #     alert.dismiss()
                #關閉阻擋頁
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 20:
                            break
                        driver.find_element_by_class_name("b-close").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.2)


                #輸入pin碼(自然人憑證)
                pinInput=driver.find_element_by_name("pin")
                pinInput.clear()
                pinInput.send_keys(pinNum)#憑證pin碼
                #輸入保險證號
                unoInput=driver.find_element_by_name("uno")
                unoInput.clear()
                unoNumber=dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']#"單位保險證號"
                unoInput.send_keys(unoNumber)
                uNoList.add(unoNumber)
                WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.NAME,'Submit')))
                time.sleep(0.5)
                #確認進入
                successful=0
                sumN=0
                while successful==0:
                    try:
                        sumN+=1
                        if sumN>100:
                            break
                        driver.find_element_by_name('Submit').click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(2)
                try:
                    WebDriverWait(driver,8).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                    time.sleep(0.4)
                    # driver.find_element_by_class_name("b-close").click()
                    successful = 0
                    sumN = 0
                    while successful == 0:
                        try:
                            sumN += 1
                            if sumN > 20:
                                break
                            driver.find_element_by_class_name("b-close").click()
                            successful = 1
                        except:
                            time.sleep(0.3)
                    time.sleep(0.4)
                except:
                    report.append("登入失敗-異動別:{}\t證號:{}\t位置{}".
                                      format(changDict[dataDict['異動別']],
                                             dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                             excelFilePath))
                    time.sleep(0.3)
                    alert=driver.switch_to_alert()
                    ###############################################判斷密碼錯誤、權限錯誤問題
                    if '讀卡機' in alert.text:
                        errorMsg="讀卡機沒插卡"
                    elif '權限' in alert.text:
                        errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
                    elif '密碼' in alert.text:
                        errorMsg="密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)

                    if errorMsg!="":
                        mailTo('ReportError',mails,errorMsg)
                        driver.quit()
                        root.destroy()
                        sys.exit("因故中斷:"+errorMsg)
                    ###############################################
                    alert.accept()
                    continue

                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
                time.sleep(0.6)
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_id("mi0,0").click()
                        successful = 1

                    except:
                        time.sleep(0.3)
                time.sleep(0.2)
                driver.find_element_by_id("mi0,5").click()
                WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'mi0,7')))
                WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'mi0,6')))
                time.sleep(0.5)

                if dataDict['異動別']=="4":
                    driver.find_element_by_id("mi0,6").click()#加保
                elif dataDict['異動別']=="2":
                    driver.find_element_by_id("mi0,7").click()#退保
                elif dataDict['異動別']=="3":
                    driver.find_element_by_id("mi0,8").click()#調保
                time.sleep(1)
                #########################################################加入生效日(連假後才需選擇)
                print(excelFilePath)
                dirName=excelFilePath.split("\\")[-2]
                try:
                    dirName=dirName.split("/")[-1]
                except:
                    pass
                print(dirName)
                if len(dirName)==7:#民國年
                    choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
                elif len(dirName)==8:#西元年
                    try:
                        choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
                    except:
                        pass
                elif len(dirName)==4:#只有月日
                    try:
                        choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
                    except:
                        pass


                try:
                    effDateOption=driver.find_element_by_name("laefDte")
                    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if choice in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                            print(date+"\n"+'*'*50)
                except:
                    pass

                #########################################################
                if "健保投保單位代碼" in dataDict:
                    driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
                else:
                    driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

                driver.find_element_by_id("uploadFile").send_keys(excelFilePath)#上傳檔案
                time.sleep(0.5)
                driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)#輸入pin
                time.sleep(0.2)
                WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'btnSaveB')))
                time.sleep(0.5)
                saveNum=0
                nnnn=0
                while saveNum==0:
                    try:
                        try:
                            driver.find_element_by_id("btnSaveB").click()
                        except:
                            driver.find_element_by_id("btnSaveA").click()
                        saveNum = 1
                    except:
                        nnnn += 1
                        time.sleep(0.3)
                        if nnnn%20==0:
                            saveNum=1

                #睡到成功為止或過了15秒
                def success():
                    try:
                        return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
                    except:
                        return False
                nsec=0
                while (not success()):
                    try:
                        if '失敗' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                            report.append("上傳失敗-異動別:{}\t證號:{}\t位置{}".
                                          format(changDict[dataDict['異動別']],
                                                 dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                                 excelFilePath))
                            try:
                                driver.find_element_by_id("mi0,53").click()
                            except:
                                driver.find_element_by_id("mi0,52").click()
                            continue
                        if '類型錯誤' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                            report.append("上傳失敗(超過30筆?)-異動別:{}\t證號:{}\t位置{}".
                                          format(changDict[dataDict['異動別']],
                                                 dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                                 excelFilePath))
                            try:
                                driver.find_element_by_id("mi0,53").click()
                            except:
                                driver.find_element_by_id("mi0,52").click()
                            continue
                    except:
                        pass
                    nsec+=0.5
                    if nsec>15:
                        break
                    time.sleep(0.5)
    #                 print(1)
                if success():
                    print("資料上傳成功-異動別:{}\t證號:{}\t位置{}".format(changDict[dataDict['異動別']],dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],excelFilePath))
    #                 report.append("資料上傳成功-異動別:{}\t證號:{}\t位置{}".
    #                                   format(changDict[dataDict['異動別']],
    #                                          dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
    #                                          excelFilePath))
					
                time.sleep(0.5)
            except Exception as e:
                report.append("上傳執行失敗-異動別:{}\t證號:{}\t位置{}\n錯誤訊息:{}\t錯誤位置:{}".
                              format(changDict[dataDict['異動別']],
                                     dataDict['勞工保險證號'] + dataDict['勞工保險證號檢查碼'],
                                     excelFilePath,e,sys.exc_info()[2].tb_lineno))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    pass
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    pass
                try:
                    driver.get(web)
                except:
                    pass
                time.sleep(0.5)
                continue
        #########################################################下載區!!!

            try:
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
                time.sleep(0.5)

                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_id("mi0,0").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.1)
                driver.find_element_by_id("mi0,10").click()
                time.sleep(0.2)
                driver.find_element_by_xpath('//*[@id="main"]/form/fieldset/table/tbody/tr[7]/td[2]/input[2]').click()
                time.sleep(0.2)
                driver.find_element_by_name('btnQuery').click()
                time.sleep(0.5)
				
                downloadDict={"4":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchPayList',
							"2":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchStopList',
							"3":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchAdjList'}
				

                driver.get(downloadDict[dataDict['異動別']])
                try:
                    time.sleep(0.2)
                    WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.NAME,'btnBackQueryForm')))
                    time.sleep(0.2)
                    if len(driver.find_elements_by_name('meslamIdOfPrint'))>0:

                        WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.NAME, 'meslamIdOfPrint')))
                        time.sleep(0.4)

                        point=driver.find_elements_by_name('meslamIdOfPrint')[0]
                        point.click()
                        fileCount+=1
                        time.sleep(0.2)

                        WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'btnPrint2')))
                        time.sleep(0.4)

                        driver.find_element_by_id('btnPrint2').click()

                    time.sleep(0.6)
                    if fileCount%50==0:
                        renamefile(path2)
                    time.sleep(0.8)
                except Exception as e:
                    time.sleep(0.5)
                    if dataDict['異動別']=='4':
                        doType = "加保"
                    elif dataDict['異動別']=='2':
                        doType = "退保"
                    elif dataDict['異動別']=='1':
                        doType = "調保"

                    report.append("下載失敗-證號:{},種類:{},錯誤訊息:{},錯誤位置:{}".format(unoNumber,doType, e,sys.exc_info()[2].tb_lineno))
                time.sleep(0.4)
                # #############################################################        
                
#                 report.append("下載成功-證號:{}".format(uNo))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    driver.find_element_by_id("mi0,52").click()
                time.sleep(1)

        
            except Exception as e:
                time.sleep(0.5)
                report.append("下載失敗-證號:{},加退調保全類,錯誤訊息:{},錯誤位置:{}".format(unoNumber,e,sys.exc_info()[2].tb_lineno))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    pass
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    pass
                try:
                    driver.get(web)
                except:
                    pass
                time.sleep(1)
        #########################################################更名區
        renamefile(path2)
        
        #########################################################
        
        driver.quit()
        e=time.time()
        costTime=round(e-b)
        report.append("\n花費時間:{}秒\n共處理{}筆資料".format(costTime,len(pathList)))
        print("\n".join(report))
        
        mailTo('Report',
               mails,
               web+"\n"+"\n".join(report))

    except Exception as e:
        if errorMsg=="":
            errorMsg=str(e)+"\n證號:{}\t位置:{}".format(unoNumber,excelFilePath)+'\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:'+str(sys.exc_info()[2].tb_lineno)+"\n"+"\n".join(report)
        
        print('出錯了RRRRRRRRRRRR'+str(e)+'\n')
        #driver.quit()
        
        mailTo('ReportError',mails,errorMsg)
        
def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
with open('defaultIO.txt') as f:
    IO=f.read()
defaultLoadDir=IO.split("\n")[0].split("=")[1].replace("\\","/")
defaultSaveDir=IO.split("\n")[1].split("=")[1].replace("\\","/")
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.insert(0, defaultLoadDir)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
e3.insert(0, defaultSaveDir)
e3.grid(row = 1, column = 1)

Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()


花費時間:13秒
共處理0筆資料


In [2]:

uNoList=set()
report=[]
changDict={"4":"加保","2":"退保","3":"調保"}

dataDir="C:\Users\andy.yuan\Desktop\開店模型討論"
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
myxlobject= xl2dict.XlToDict()
fileCount=0
for excelFilePath in pathList:
    try:
        #得到excel內容成dict
#         dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#         time.sleep(0.2)
#         #進入勞保局
# #         WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
# #         time.sleep(0.4)
#     #     alert=driver.switch_to_alert()
#     #     alert.dismiss()
#         #關閉阻擋頁
#         successful = 0
#         sumN = 0
# #         while successful == 0:
# #             try:
# #                 sumN += 1
# #                 if sumN > 20:
# #                     break
# #                 driver.find_element_by_class_name("b-close").click()
# #                 successful = 1
# #             except:
# #                 time.sleep(0.3)
# #         time.sleep(0.2)


#         #輸入pin碼(自然人憑證)
#         pinInput=driver.find_element_by_name("pin")
#         pinInput.clear()
#         pinInput.send_keys(pinNum)#憑證pin碼
#         #輸入保險證號
#         unoInput=driver.find_element_by_name("uno")
#         unoInput.clear()
#         unoNumber=dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']#"單位保險證號"
#         unoInput.send_keys(unoNumber)
#         uNoList.add(unoNumber)
#         WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.NAME,'Submit')))
#         time.sleep(0.5)
#         #確認進入
#         successful=0
#         sumN=0
#         while successful==0:
#             try:
#                 sumN+=1
#                 if sumN>100:
#                     break
#                 driver.find_element_by_name('Submit').click()
#                 successful = 1
#             except:
#                 time.sleep(0.3)
#         time.sleep(2)
#         try:
#             WebDriverWait(driver,8).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#             time.sleep(0.4)
#             # driver.find_element_by_class_name("b-close").click()
#             successful = 0
#             sumN = 0
#             while successful == 0:
#                 try:
#                     sumN += 1
#                     if sumN > 20:
#                         break
#                     driver.find_element_by_class_name("b-close").click()
#                     successful = 1
#                 except:
#                     time.sleep(0.3)
#             time.sleep(0.4)
#         except:
#             report.append("登入失敗-異動別:{}\t證號:{}\t位置{}".
#                               format(changDict[dataDict['異動別']],
#                                      dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
#                                      excelFilePath))
#             time.sleep(0.3)
#             alert=driver.switch_to_alert()
#             ###############################################判斷密碼錯誤、權限錯誤問題
#             if '讀卡機' in alert.text:
#                 errorMsg="讀卡機沒插卡"
#             elif '權限' in alert.text:
#                 errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
#             elif '密碼' in alert.text:
#                 errorMsg="密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)

#             if errorMsg!="":
#                 mailTo('ReportError',mails,errorMsg)
#                 driver.quit()
#                 root.destroy()
#                 sys.exit("因故中斷:"+errorMsg)
#             ###############################################
#             alert.accept()
#             continue

#         WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
#         time.sleep(0.6)
#         successful = 0
#         sumN = 0
#         while successful == 0:
#             try:
#                 sumN += 1
#                 if sumN > 100:
#                     break
#                 driver.find_element_by_id("mi0,0").click()
#                 successful = 1

#             except:
#                 time.sleep(0.3)
#         time.sleep(0.2)
#         driver.find_element_by_id("mi0,5").click()
#         WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'mi0,7')))
#         WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID, 'mi0,6')))
#         time.sleep(0.5)

#         if dataDict['異動別']=="4":
#             driver.find_element_by_id("mi0,6").click()#加保
#         elif dataDict['異動別']=="2":
#             driver.find_element_by_id("mi0,7").click()#退保
#         elif dataDict['異動別']=="3":
#             driver.find_element_by_id("mi0,8").click()#調保
#         time.sleep(1)
        #########################################################加入生效日(連假後才需選擇)
        print(excelFilePath)
        dirName=excelFilePath.split("\\")[-2]
        try:
            dirName=dirName.split("/")[-1]
        except:
            pass
        print(dirName)
        if len(dirName)==7:#民國年
            choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
        elif len(dirName)==8:#西元年
            try:
                choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
            except:
                pass
        elif len(dirName)==4:#只有月日
            try:
                choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
            except:
                pass
            

SyntaxError: unexpected EOF while parsing (<ipython-input-2-67294e6f8293>, line 146)

In [18]:
def doAddPopChangeAssure():
    dataDir=e1.get()
    print(dataDir)
    pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
    myxlobject= xl2dict.XlToDict()
    fileCount=0
    for excelFilePath in pathList:
        print(excelFilePath)
        dirName=excelFilePath.split("\\")[-2]
        print(1,dirName)
        try:
            dirName=dirName.split("/")[1]
        except:
            pass
        print(2,dirName)
        if len(dirName)==7:#民國年
            choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
        elif len(dirName)==8:#西元年
            try:
                choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
            except:
                pass
        elif len(dirName)==4:#只有月日
            try:
                choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
            except:
                pass
        print(choice)
        
def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
with open('defaultIO.txt') as f:
    IO=f.read()
defaultLoadDir=IO.split("\n")[0].split("=")[1].replace("\\","/")
defaultSaveDir=IO.split("\n")[1].split("=")[1].replace("\\","/")
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.insert(0, defaultLoadDir)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
e3.insert(0, defaultSaveDir)
e3.grid(row = 1, column = 1)

Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()

D:\保險申報檔案\保險申報檔案
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_加1.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_加2二合一.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_調1二合一.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_調2.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_退1.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801_退2二合一.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\保證改掉sample.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\改第二保險證號.xls
1 1070622
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802_加1.xls
1 1090313
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802_加2二合一.xls
1 1090313
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802_調1二合一.xls
1 1090313
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802_調2.xls
1 1090313
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802_退1.xls
1 1090313

In [40]:
dataDir="D:\保險申報檔案\保險申報檔案"
print(dataDir)
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
myxlobject= xl2dict.XlToDict()
fileCount=0
for excelFilePath in pathList:
    print(excelFilePath)
    dirName=excelFilePath.split("\\")[-2]
    print(1,dirName)
    layer=-3
    while not (len(dirName)==4 or len(dirName)==7 or len(dirName)==8):
        dirName=excelFilePath.split("\\")[layer]
        layer-=1
        print(dirName)
        print(len(dirName))
    try:
        dirName=dirName.split("/")[1]
    except:
        pass
    print(2,dirName)
    if len(dirName)==7:#民國年
        choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
    elif len(dirName)==8:#西元年
        try:
            choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
        except:
            pass
    elif len(dirName)==4:#只有月日
        try:
            choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
        except:
            pass
    print(choice)

D:\保險申報檔案\保險申報檔案
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_加1.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_加2二合一.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_調1二合一.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_調2.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_退1.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\11801_退2二合一.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\保證改掉sample.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1070622\11801\改第二保險證號.xls
1 11801
1070622
7
2 1070622
107/06/22
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802\11802_加2二合一.xls
1 11802
1090313
7
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802\11802_調1二合一.xls
1 11802
1090313
7
2 1090313
109/03/13
D:\保險申報檔案\保險申報檔案\1060613\1090313\11802\11802_調2.xls
1 11802
1090313

In [29]:
dirName

'D:'

In [31]:
layer

-8

In [32]:
dirName

'D:'